<a href="https://colab.research.google.com/github/Jmat201/LLMs/blob/main/TECH%2016_Project_course_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install sentence-transformers
!pip install langchain pypdf langchain-openai #tiktoken chromadb
!pip install nest-asyncio
!pip install llama-index --upgrade
!pip install google-api-python-client
!pip install streamlit PyPDF2 llama-index
%pip install llama-index llama-index-readers-web
%pip install --q crewai
%pip install --q -U duckduckgo-search
%pip install langchain-openai
import nest_asyncio
nest_asyncio.apply()


In [ ]:
import os
from langchain_openai import OpenAI, ChatOpenAI
from crewai import Agent, Task, Crew, Process
from google.colab import userdata
from langchain.tools import DuckDuckGoSearchRun
import streamlit as st
import os
import PyPDF2
import tempfile
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.web import SimpleWebPageReader

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [ ]:

# see if pypdf reader needed

llm = OpenAI(temperature=0.1, openai_api_key=userdata.get('open_ai_key_1'),model = 'gpt-4-turbo-preview')
# setting environment variable for open AI key
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key_1')

web_search_tool = DuckDuckGoSearchRun()

def extract_text_from_pdf(uploaded_file):
    # Create a PdfReader object
    pdf_reader = PyPDF2.PdfReader(uploaded_file)

    # Extract text from each page
    text = ''.join(page.extract_text() for page in pdf_reader.pages)

    return text

    # Define the URLs of the websites you want to read
website_urls = [
    "https://www.edx.org/search?tab=course",
    "https://www.coursera.org/directory/courses",
    "https://maven.com/courses",
    "https://bulletin.gwu.edu/courses/"
]
web_documents = SimpleWebPageReader().load_data(urls=website_urls)


In [ ]:
document_tool = {
    "name": "Document Analysis Tool",
    "data": SimpleDirectoryReader("./").load_data("resume.pdf"),
    "description": "Tool for analyzing resume PDF files."
}


Loading files: 100%|██████████| 1/1 [00:00<00:00,  3.74file/s]


In [ ]:
question_to_answer = "Which course should the user do?"
#website = "https://www.linkedin.com/jobs/search/?currentJobId=4012443092&keywords=Data%20scientist&origin=JOB_SEARCH_PAGE_JOB_FILTER&refresh=true&sortBy=DD"


In [ ]:
#Career consultant or HR consultant - need to change. Changed to technical roles specifically.

text = SimpleDirectoryReader("./").load_data("resume.pdf")


career_consultant = Agent(
    role="Human Resource Consultant",
    goal="Analyze the user's resume and identify 5 technical skills that the user has.",
    backstory="You are a serior career consultant skilled in recruiting for technical roles. ",
    llm=llm,
    tools=[],
    verbose=True,
    allow_delegation=False
)

target_role = input("Please enter the target role you are interested in: ")


# Create tasks for your agents -- ONLY TECHNICAL SKILLS
task1 = Task(
  description=f"""List all technical skills mentioned in the resume : {text}""",
  agent= career_consultant,
  expected_output=f"""A list of all the technical skills listed in the resume: {text}"""
)



Loading files: 100%|██████████| 1/1 [00:00<00:00,  3.29file/s]


Please enter the target role you are interested in: data scientist


In [ ]:

Research_Analyst_1 = Agent(
    role="Research Analyst ",
    goal=f"Search and write a list of 5 skills needed for '{target_role}'. Be concise.",
    backstory="You are a skilled reseach analyst with an IQ of 125 and have extensive knowledge of how to search for all technical skills required for a job.",
    llm=llm,
    tools=[web_search_tool],
    verbose=True,
    allow_delegation=False
)

# Task - reasearcher finds job skills for target role
task2 = Task(
        description=f"""list five skills required  for the role: '{target_role}'.""",
        agent=Research_Analyst_1,
        expected_output="""A list of 5 job skills required for the specified target role.""",
        context = [task1]
    )


In [ ]:

Research_Analyst_2 = Agent(
    role="Research Analyst",
    goal=f"Search and write a list of 5 courses based on skills in :'{task2}'. Be as detailed as you need.",
    backstory="You are a skilled reseach analyst with an IQ of 125 and have extensive knowledge of finding the best courses that teach a set of skills.",
    llm=llm,
    tools=[web_search_tool], #web_search_tool
    verbose=True,
    allow_delegation=False
)


# Task 3 - Finding courses that teach those skills from web_documents(weblinks_url) - removed web links
task3 = Task(
        description=f"""list 5 courses that teach all of the skills :'{task2}'.""",
        agent=Research_Analyst_2,
        expected_output="""A list of courses that teach the specified technical skills .""",
        context = [task2]
    )


In [ ]:
Research_Analyst_3 = Agent(
    role="Critic",
    goal=f"Compare the suggested courses in :'{task3}' and the skills in: '{task2}' and critic if they match. suggest which skills do not match the give courses recommended.",
    backstory="You are a skilled reseach analyst with an IQ of 125 and have extensive knowledge of how to matching skills to courses .",
    llm=llm,
    tools=[], #web_search_tool
    verbose=True,
    allow_delegation=False
)

# Task 4- critic non-matching courses and skills
task4 = Task(
        description=f"""Using the courses recommended 2 courses to answer the question:'{question_to_answer}'""",
        agent=Research_Analyst_3,
        expected_output="""List two courses that teach the specified technical skills.""",
        context = [task3]
    )

In [ ]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[Research_Analyst_1,Research_Analyst_2], # career_consultant,  , Research_Analyst_3
  tasks=[task2, task3], # , task3, task4
  verbose=True, # You can set it to 1 or 2 to different logging levels,
)

In [ ]:
crew

Crew(id=36ba4b63-7d9d-4d89-9005-28e5969bafde, process=sequential, number_of_agents=2, number_of_tasks=2)

In [ ]:
result = crew.kickoff()

 [2024-08-29 14:19:37][DEBUG]: == Working Agent: Research Analyst 
 [2024-08-29 14:19:37][INFO]: == Starting Task: Analyze LinkedIn job skills for the role of 'data scientist' and list the required skills.


> Entering new CrewAgentExecutor chain...


> Finished chain.
 [2024-08-29 14:20:18][DEBUG]: == [Research Analyst ] Task output: Agent stopped due to iteration limit or time limit.


 [2024-08-29 14:20:18][DEBUG]: == Working Agent: Research Analyst
 [2024-08-29 14:20:18][INFO]: == Starting Task: list 5 courses that teach all of the skills :'used_tools=0 tools_errors=0 delegations=0 i18n=I18N(prompt_file=None) name=None prompt_context=None description="Analyze LinkedIn job skills for the role of 'data scientist' and list the required skills." expected_output='A list of 5 job skills required for the specified target role.' config=None callback=None agent=Agent(role=Research Analyst , goal=Search and write a list of 5 skills needed for 'data scientist'. Be concise., backstory=You are 

In [ ]:
from IPython.display import display, Markdown
display(Markdown(result.raw))

Agent stopped due to iteration limit or time limit.